In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/spotify-south-africa-top-200-weekly-streams/SA_Spotify_data2.csv')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.templates.default = "presentation"
style.use('fivethirtyeight')
sns.set_context("poster")
sns.color_palette("Paired")
%matplotlib inline

In [ ]:
df.head()

In [ ]:
df['Week_BD'] = df['Week'].apply((lambda x: x[-10:]))

In [ ]:
df['Week_BD'] = pd.to_datetime(df['Week_BD'])

In [ ]:
df.info()

In [ ]:
# Number of unique songs and artists
print(df['Artist'].nunique())
df['Track Name'].nunique()

In [ ]:
df.corr()

Position and streams, obviously

Length and pop, instrumentalness, loudness

acousticness and energy

energy and loudness

instrumentalness and loudness

In [ ]:
top_guy = df[df['Position']==1]
top_guy


Positions are kept for concurrent weeks

## Data Analysis

We can also group by the week. find average streams

In [ ]:
streams_per_week_avg = df.groupby(['Week_BD'], as_index=False)['Streams'].mean()
streams_per_week_sum = df.groupby(['Week_BD'], as_index=False)['Streams'].sum()

In [ ]:
streams_per_week_avg.sort_values(by='Streams').tail()

In [ ]:
streams_per_week_sum.sort_values(by='Streams').tail()

In [ ]:
#Most common genre
df['artist_top_genre'].value_counts(normalize=True, ascending=False).head(10)

In [ ]:
# Most Streams in Week
df.groupby(['Artist', 'Track Name'],as_index=False)['Streams'].max().sort_values(by='Streams', ascending=False).head(10)

In [ ]:
# Most popular artist
df['Artist'].value_counts(ascending=False).head(10)

In [ ]:
# popular albums
df['album'].value_counts(ascending=False).head(10)

In [ ]:
# Most common artist in the Top 30 streams per week
df1 = df[df['Position'] <= 30]
df1['Artist'].value_counts(ascending=False).head(15)

In [ ]:
# Most common song in the Top 30 streams per week
df1['Track Name'].value_counts(ascending=False).head(15)

In [ ]:
# Oldest song on the chart
df[df['release_date'] == df['release_date'].min()]

In [ ]:

px.box(df, y="danceability", points='all', hover_name='Track Name', hover_data=['Artist'])

In [ ]:
# Removing duplicate songs from different weeks
no_dupli = df.copy()
no_dupli.drop(columns=['Streams','Position','Week','Week_BD'], inplace=True)
no_dupli.drop_duplicates(inplace=True)

In [ ]:
no_dupli.info()

In [ ]:
# Most popular songs (Popularity is based on the entire Spotify database)
no_dupli.sort_values(by=['popularity'], ascending=False).head(10)

In [ ]:
# Popular genres for artists
no_dupli.groupby(['artist_top_genre'],as_index=False)['Artist'].count().sort_values('Artist', ascending=False).head(15)

In [ ]:
# Isolating the top 10 genres
df_genre = df[df['artist_top_genre'].isin(["dance pop", "pop","afro house","chicago rap","alternative r&b","melodic rap","dfw rap","rap","canadian hip hop"])]

In [ ]:
px.box(df_genre, x='artist_top_genre', color='artist_top_genre',y="popularity", points='all', hover_name='Track Name', hover_data=['Artist'])

In [ ]:
# Top genres in the top 30 streams per week
df_genre1 = df1[df1['artist_top_genre'].isin(["dance pop", "pop","afro house","chicago rap","brooklyn drill","melodic rap","dfw rap","electropop","canadian contemporary r&b"])]

In [ ]:
px.box(df_genre1, x='artist_top_genre', color='artist_top_genre',y="popularity", points='all', hover_name='Track Name', hover_data=['Artist'])

In [ ]:
plt.figure(figsize=(16, 6))
mask = np.triu(np.ones_like(no_dupli.corr(), dtype=np.bool))
heatmap = sns.heatmap(no_dupli.corr(), mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Audio Features Correlation Heatmap', fontdict={'fontsize':18}, pad=16)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(18,10))
plt.tight_layout()
axes[0][0].plot(no_dupli['length'].value_counts().sort_index())
axes[0][0].set_title('Duration(ms)')
axes[0][1].plot(no_dupli['energy'].value_counts().sort_index())
axes[0][1].set_title('energy')
axes[0][2].plot(no_dupli['danceability'].value_counts().sort_index())
axes[0][2].set_title('danceability')
axes[0][3].plot(no_dupli['loudness'].value_counts().sort_index())
axes[0][3].set_title('loudness(dB)')
axes[0][4].plot(no_dupli['liveness'].value_counts().sort_index())
axes[0][4].set_title('liveness')
axes[1][0].plot(no_dupli['tempo'].value_counts().sort_index())
axes[1][0].set_title('tempo')
axes[1][1].plot(no_dupli['instrumentalness'].value_counts().sort_index())
axes[1][1].set_title('instrumentalness')
axes[1][2].plot(no_dupli['acousticness'].value_counts().sort_index())
axes[1][2].set_title('acousticness')
axes[1][3].plot(no_dupli['speechiness'].value_counts().sort_index())
axes[1][3].set_title('speechiness')
axes[1][4].plot(no_dupli['popularity'].value_counts().sort_index())
axes[1][4].set_title('popularity')

In [ ]:
f, axes = plt.subplots(2, 5, figsize=(20, 12))
sns.despine(left=True)
sns.boxplot(x=no_dupli.instrumentalness, color="r", ax=axes[0, 0])
sns.boxplot(x=df.energy,color="b", ax=axes[0, 1])
sns.boxplot(x=df.danceability, color="r", ax=axes[0, 2])
sns.boxplot(x=df.loudness, color="b", ax=axes[0, 3])
sns.boxplot(x=df.liveness, color="r", ax=axes[0, 4])
sns.boxplot(x=df.tempo, color="b", ax=axes[1, 0])
sns.boxplot(x=df.length, color="r", ax=axes[1, 1])
sns.boxplot(x=df.acousticness, color="b", ax=axes[1, 2])
sns.boxplot(x=df.speechiness, color="r", ax=axes[1, 3])
sns.boxplot(x=df.popularity, color="b", ax=axes[1, 4])

In [ ]:
fig = px.scatter(df, x="danceability", y="energy", color='energy', hover_name='Track Name', hover_data=['Artist','artist_top_genre'])
fig.show()

In [ ]:
px.box(df_genre1, x='artist_top_genre', color='artist_top_genre',y="speechiness", points='all', hover_name='Track Name', hover_data=['Artist'])

In [ ]:
px.box(df_genre1, x='artist_top_genre', color='artist_top_genre',y="tempo", points='all', hover_name='Track Name', hover_data=['Artist'])

In [ ]:
#calculate the average of the popularity and the size of each genre 
genre_popularity = df.groupby(["artist_top_genre"], as_index=False)
genre_popularity = genre_popularity.agg({'popularity':[np.mean, np.size]})
genre_popularity.columns = list(map(''.join, genre_popularity.columns.values))
#sort
genre_popularity = genre_popularity.sort_values(by=["popularitymean"], ascending = False)

genre_popularity.head(10)

In [ ]:
fig = px.scatter(genre_popularity, y="popularitymean", x="popularitysize", hover_name='artist_top_genre', hover_data=['artist_top_genre'])
fig.show()

## Clustering

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
rs = 123
np.random.seed(rs)

def pca_plot(df, classes=None):

    # Scale data for PCA
    scaled_df = StandardScaler().fit_transform(df)
    
    # Fit the PCA and extract the first two components
    pca_results = PCA().fit_transform(scaled_df)
    pca1_scores = pca_results[:,0]
    pca2_scores = pca_results[:,1]
    
    # Sort the legend labels
    if classes is None:
      hue_order = None
      n_classes = 0
    elif str(classes[0]).isnumeric():
      classes = ['Cluster {}'.format(x) for x in classes]
      hue_order = sorted(np.unique(classes))
      n_classes = np.max(np.unique(classes).shape)
    else:
      hue_order = sorted(np.unique(classes))
      n_classes = np.max(np.unique(classes).shape)

    # Plot the first two principal components
    plt.figure(figsize=(8.5,8.5))
    plt.grid()
    sns.scatterplot(pca1_scores, pca2_scores, s=50, hue=classes,
                    hue_order=hue_order, palette='Set1')
    plt.xlabel("Principal Component {}".format(1))
    plt.ylabel("Principal Component {}".format(2))
    plt.title('Principal Component Plot')
    plt.show()

In [ ]:

audio_feature_cols = ['danceability', 'energy', 'loudness', 
                      'speechiness', 'acousticness', 'instrumentalness',
                      'liveness', 'tempo', 'length',
                      'time_signature']

In [ ]:
# Plot the principal component analysis results
pca_plot(no_dupli[audio_feature_cols])

In [ ]:
scaled_df = StandardScaler().fit_transform(no_dupli[audio_feature_cols])
# Calculate the Total Distance for each value of k between 1 and 10
scores = []
for i in range(1,11):
    fit_k = KMeans(n_clusters=i, n_init=5, random_state=rs).fit(scaled_df)
    scores.append(fit_k.inertia_)

# Plot this in an elbow plot
plt.figure(figsize=(11,8.5))
sns.lineplot(range(1,11), scores, palette=sns.color_palette('Set1'))
plt.xlabel('Number of clusters $k$')
plt.ylabel('Total Point to Centroid Distance')
plt.grid()
plt.title('The Elbow Method showing the optimal $k$')
plt.show()

In [ ]:
best_k = 3
best_cluster_results = KMeans(n_clusters=best_k, n_init=25, random_state=rs).fit(scaled_df)

# Save the cluster labels in our dataframe
no_dupli['best_cluster'] = ['Cluster ' + str(i) for i in best_cluster_results.labels_]

# Show a PCA plot of the clusters
pca_plot(no_dupli[audio_feature_cols], classes=no_dupli['best_cluster'])

In [ ]:
no_dupli.groupby(['best_cluster'],as_index=False).size()

In [ ]:
# Plot the distribution of audio features by cluster
g = sns.pairplot(no_dupli, hue="best_cluster",
                 vars=['danceability', 'energy', 'loudness', 'speechiness', 'tempo'],
                 hue_order=sorted(no_dupli.best_cluster.unique()), palette='Set1')
g.fig.suptitle('Distribution of Audio Features by Cluster', y=1.05)
plt.show()

In [ ]:
c = no_dupli.groupby(['best_cluster','artist_top_genre','Track Name'], as_index=False)['popularity'].mean().sort_values(by='popularity').head(20)

In [ ]:

px.sunburst(c, path=['best_cluster','artist_top_genre', 'Track Name'], values='popularity', title= 'Sunburst chart for top songs',
           color_discrete_sequence=px.colors.qualitative.Set2, branchvalues='total', width=2000, height = 1000)
